In [1]:
import sys
sys.path.insert(0,"/home/albert7a/lib/python")


## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
import time
from dask.diagnostics import ProgressBar
from datetime import date

In [2]:
latmin=-1.5
latmax=5
lonmin=35
lonmax=40
datemin='2012-09-01T00:00'
datemax='2012-09-30T23:59'

In [4]:
tfiles="/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y????m??d??.1d_gridT.nc"
dsN = xr.open_mfdataset(tfiles,concat_dim='time_counter',decode_times=False, chunks={'deptht':1 ,'time_counter':10})

latN = dsN.nav_lat
lonN = dsN.nav_lon
timN = dsN.time_counter

lamin=np.nanmin(latN.values)
lamax=np.nanmax(latN.values)
lomin=np.nanmin(lonN.values)
lomax=np.nanmax(lonN.values)
tmin=np.min(timN.values)
tmax=np.max(timN.values)

print 'lamin=',lamin,'lamax=',lamax,'lomin=',lomin,'lomax=',lomax,'tmin=',tmin,'tmax=',tmax

datetmin=pd.to_datetime(datemin)
datetmax=pd.to_datetime(datemax)
ttmin=datetime.datetime(int(datetmin.strftime('%Y')),int(datetmin.strftime('%m')),int(datetmin.strftime('%d')),int(datetmin.strftime('%H')),int(datetmin.strftime('%M')))
ttmax=datetime.datetime(int(datetmax.strftime('%Y')),int(datetmax.strftime('%m')),int(datetmax.strftime('%d')),int(datetmax.strftime('%H')),int(datetmax.strftime('%M')))
tsecmin=(ttmin-datetime.datetime(1958,1,1,0,0)).total_seconds()
tsecmax=(ttmax-datetime.datetime(1958,1,1,0,0)).total_seconds()

print 'tsecmin=',tsecmin,'tsecmax=',tsecmax
if (lamin < latmin < lamax) & (lamin < latmax < lamax) & (lomin < lonmin < lomax) & (lomin < lonmax < lomax) & (tmin < tsecmin < tmax) & (tmin < tsecmax < tmax):
        print "selected area falls within NATL60-CJM165 boundaries, the program is proceeding"
else:
        print "selected area does not fall within NATL60-CJM165 boundaries, the program is stopping"



 lamin= 26.5648 lamax= 67.3871 lomin= -86.255 lomax= 17.7867 tmin= 1719835200.0 tmax= 1722427200.0
tsecmin= 1725148800.0 tsecmax= 1727740740.0
selected area does not fall within NATL60-CJM165 boundaries, the program is stopping


In [5]:
yearmin=datemin[0:4]
monthmin=datemin[5:7]
yearmax=datemax[0:4]
monthmax=datemax[5:7]


In [6]:
diren4="/media/extra/DATA/ARGO/"
list_filesEN4=[]
if yearmin == yearmax:
    for m in np.arange(int(monthmin),int(monthmax)+1):
        if m < 10:
            list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+'0'+str(m)+'.nc')
        else:
            list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+str(m)+'.nc')
else:
    for m in np.arange(int(monthmin),13):
        if m < 10:
            list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+'0'+str(m)+'.nc')
        else:
            list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+str(m)+'.nc')
    for m in np.arange(1,int(monthmax)+1):
        if m < 10:
            list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmax+'0'+str(m)+'.nc')
        else:
            list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmax+str(m)+'.nc')
print list_filesEN4

['EN.4.2.0.f.profiles.g10.201209.nc']


In [143]:
print tsecmin,tsecmax

1721606400.0 1721779140.0


In [12]:
for f in np.arange(len(list_filesEN4)):
    fileEN4=list_filesEN4[f]
    tfileEN4=diren4+fileEN4
    print tfileEN4
    !ls $tfileEN4
    dsen4=xr.open_dataset(tfileEN4)
    laten4=dsen4['LATITUDE']
    lonen4=dsen4['LONGITUDE']
    dayen4=dsen4['JULD']
    refen4=dsen4['DC_REFERENCE']

    indz=np.where((lonmin<lonen4.values)&(lonen4.values<lonmax)&(latmin<laten4.values)&(laten4.values<latmax))
    prof_zone=[]
    tsecen4z=[]
    print len(indz[0])
    
    for ref in np.arange(len(indz[0])):
            dateen4= pd.to_datetime(str(dayen4[indz[0][ref]].values))
            ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
            tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()
            if (tsecen4 < tsecmax) & (tsecen4 > tsecmin):
                    print str(dayen4[indz[0][ref]].values),tsecen4
                    prof_zone.append(indz[0][ref])
                    tsecen4z.append(tsecen4)

    print len(prof_zone)
    
    for ref in np.arange(len(prof_zone)):
            if (f == 0) & (ref == 0):
                    dictyml={str(refen4[prof_zone[ref]].values):{'reference':str(refen4[prof_zone[ref]].values),'file':fileEN4,'profile no':int(prof_zone[ref]),'latitude':float(laten4[prof_zone[ref]].values),'longitude':float(lonen4[prof_zone[ref]].values),'date':str(dayen4[prof_zone[ref]].values)}}
            else:
                    up={str(refen4[prof_zone[ref]].values):{'reference':str(refen4[prof_zone[ref]].values),'file':fileEN4,'profile no':int(prof_zone[ref]),'latitude':float(laten4[prof_zone[ref]].values),'longitude':float(lonen4[prof_zone[ref]].values),'date':str(dayen4[prof_zone[ref]].values)}}
                    dictyml.update(up)


    print dictyml[str(refen4[prof_zone[0]].values)]

    with io.open(jsonfile, 'w', encoding='utf8') as outfile:
            outfile.write(unicode(json.dumps(dictyml, sort_keys=True,indent=4, separators=(',', ': '))))



/media/extra/DATA/ARGO/EN.4.2.0.f.profiles.g10.201209.nc
/media/extra/DATA/ARGO/EN.4.2.0.f.profiles.g10.201209.nc
0
0


NameError: name 'dictyml' is not defined

In [85]:
dictyml

{1: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02752', 2040],
 2: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02753', 2041],
 3: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02754', 2055],
 4: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02755', 2065],
 5: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02756', 2071],
 6: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02757', 2083],
 7: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02758', 44904],
 8: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02759', 44907],
 9: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02760', 44908],
 10: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02761', 44910]}

In [93]:
import yaml
import io

# Define data
data = {1: ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE',222], 2 : ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE',222]}


# Write YAML file
with io.open('data.yaml', 'w', encoding='utf8') as outfile:
    yaml.dump(data, outfile, default_flow_style=False, allow_unicode=True)



In [66]:
# Read YAML file
with open("data.yaml", 'r') as stream:
    data_loaded = yaml.load(stream)

print(data == data_loaded)

True


In [67]:
print data_loaded

{1: ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE', 222], 2: ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE', 222]}


In [69]:
data={1: ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE',222]}
data2={2: ['EN.4.2.0.f.profiles.g10.201204.nc', 'DC_REFERENCE',223]}
data.update(data2)

In [70]:
print data

{1: ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE', 222], 2: ['EN.4.2.0.f.profiles.g10.201204.nc', 'DC_REFERENCE', 223]}


In [71]:
np.arange(12)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [87]:
print data

{1: ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE', 222], 2: ['EN.4.2.0.f.profiles.g10.201204.nc', 'DC_REFERENCE', 223]}


In [88]:
print dictyml

{1: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02752', 2040], 2: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02753', 2041], 3: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02754', 2055], 4: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02755', 2065], 5: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02756', 2071], 6: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02757', 2083], 7: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02758', 44904], 8: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02759', 44907], 9: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02760', 44908], 10: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02761', 44910]}


In [90]:
format(data)

"{1: ['EN.4.2.0.f.profiles.g10.201206.nc', 'DC_REFERENCE', 222], 2: ['EN.4.2.0.f.profiles.g10.201204.nc', 'DC_REFERENCE', 223]}"

In [91]:
format(dictyml)

"{1: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02752', 2040], 2: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02753', 2041], 3: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02754', 2055], 4: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02755', 2065], 5: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02756', 2071], 6: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02757', 2083], 7: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02758', 44904], 8: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02759', 44907], 9: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02760', 44908], 10: ['EN.4.2.0.f.profiles.g10.201207.nc', ' A20120701-02761', 44910]}"

In [109]:
dictyml={int(ref+1):[fileEN4,str(refen4[0].values),int(prof_zone[0])]}
with io.open('data.yaml', 'w', encoding='utf8') as outfile:
            yaml.dump(dictyml, outfile, default_flow_style=False, allow_unicode=True)



In [114]:
refen4[ref].data

array(' A20120701-02761', 
      dtype='|S16')

In [124]:
str(dayen4[0].values)

'2012-07-01T23:53:41.000000000'

In [126]:
datemin='2012-07-22'
datetmin=pd.to_datetime(datemin)
ttmin=datetime.datetime(int(datetmin.strftime('%Y')),int(datetmin.strftime('%m')),int(datetmin.strftime('%d')),int(datetmin.strftime('%H')),int(datetmin.strftime('%M')))
tsecmin=(ttmin-datetime.datetime(1958,1,1,0,0)).total_seconds()
print tsecmin

1721606400.0


In [127]:
datemin='2012-07-01'
datetmin=pd.to_datetime(datemin)
ttmin=datetime.datetime(int(datetmin.strftime('%Y')),int(datetmin.strftime('%m')),int(datetmin.strftime('%d')),int(datetmin.strftime('%H')),int(datetmin.strftime('%M')))
tsecmin=(ttmin-datetime.datetime(1958,1,1,0,0)).total_seconds()
print tsecmin

1719792000.0


In [128]:
print str(dayen4[indz[0][0]].values)
dateen4= pd.to_datetime(str(dayen4[indz[0][0]].values))
ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()
print tsecen4

2012-07-01T06:06:03.000000000
1719813960.0
